In [92]:
import pandas as pd

C = pd.read_stata(
    "/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/00_source_data/0405_sec_c0.dta"
)
print(C.shape)

(91319, 21)


In [93]:
C.columns

Index(['hhcode', 'sec', 'msno', 'scq01', 'scq02', 'scq03', 'scqo4', 'scq05',
       'scq06', 'scq07', 'scq08a', 'scq08b', 'scq09a', 'scq09b', 'psu',
       'quarter', 'region', 'psu_hies', 'hhno', 'hhcode_hies', 'province'],
      dtype='object')

In [94]:
# Reduce to the questions of interest (4 questions):

C["ever_admitted"] = C["scq03"]
C["max_level_acheived"] = C["scqo4"]
C["currently_enrolled"] = C["scq05"] # IMPORTANT!!! (counts of enrrollment)
C["why_not"] = C["scq09a"]

C_relevant = C[
    [
        "hhcode",
        "msno",
        "ever_admitted",
        "max_level_acheived",
        "currently_enrolled",
        "why_not",
        "region",
        "province"
    ]
].copy()


In [95]:
C_relevant

,hhcode,msno,ever_admitted,max_level_acheived,currently_enrolled,why_not,region,province
0,1.001100e+09,1,yes,class 10,no,2 education completed,urban,punjab
1,1.001100e+09,2,yes,cl;ass 8,no,2 education completed,urban,punjab
2,1.001100e+09,3,yes,17: ma/msc,no,2 education completed,urban,punjab
3,1.001100e+09,4,yes,class 10,yes,NaN,urban,punjab
4,1.001100e+09,1,yes,12: ba/bsc,no,10 employment,urban,punjab
...,...,...,...,...,...,...,...,...
91314,4.262002e+09,6,no,NaN,NaN,15 other,rural,balochistan
91315,4.262002e+09,1,no,NaN,NaN,10 employment,rural,balochistan
91316,4.262002e+09,2,no,NaN,NaN,13 parent did not approve,rural,balochistan
91317,4.262002e+09,3,no,NaN,NaN,13 parent did not approve,rural,balochistan


In [96]:
B = pd.read_stata(
    "/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/00_source_data/0405_sec_b0.dta"
)
B.sample(5)

# Change the names of the questions for readability
B['sex'] = B['sbq01']
B['age'] = B['sbq04']
B['marital_status'] = B['sbq05']

B_relevant = B[['hhcode', 'msno', 'age', 'sex', 'marital_status']].copy()

In [97]:
B.sample(5)

,hhcode,sec,msno,sbq01,sbq02,sbq03,sbq04,sbq05,weight,psu,quarter,region,psu_hies,hhno,hhcode_hies,province,sex,age,marital_status
96583,4.162001e+09,B,1,male,present,1:self,47,2:currently married,367.980011,41620006,4,rural,40420403,4,4.042040e+09,balochistan,male,47,2:currently married
37400,1.322002e+09,B,2,male,present,3:son/daughter,38,2:currently married,2705.428955,13220018,2,rural,13220201,9,1.322020e+09,punjab,male,38,2:currently married
99491,4.222001e+09,B,4,male,present,3:son/daughter,1,1:never married,359.971008,42220013,2,rural,40520204,13,4.052020e+09,balochistan,male,1,1:never married
63668,2.162002e+09,B,2,female,present,2: husband/wife,50,2:currently married,1328.099976,21620018,2,rural,21620201,11,2.162020e+09,sindh,female,50,2:currently married
7312,1.061001e+09,B,8,male,present,3:son/daughter,6,1:never married,1664.115967,10610012,2,urban,10210202,12,1.021020e+09,punjab,male,6,1:never married


In [98]:
# Made the Household Code column a str type
B["hhcode"] = B["hhcode"].astype("str")
C["hhcode"] = C["hhcode"].astype("str")

In [99]:
# MERGE
years04_05 = pd.merge(B_relevant, C_relevant, on=["hhcode", 'msno'], how='inner', indicator=False)

# Evaluate merge 
#years04_05._merge.value_counts()

In [100]:
# Select people in school enrollment age (I would say 4-15 years old)

years04_05 = years04_05.loc[(years04_05["age"] > 4)&(years04_05["age"] <= 15)]
years04_05['year']=2004

In [101]:
# Change dtype to string (hhcode)
years04_05["hhcode"] = years04_05["hhcode"].astype("str")

In [102]:
#years04_05['subprovince']=years04_05['hhcode'].str.extract(r'(^w{3})')
years04_05['subprovince code']= years04_05['hhcode'].apply(lambda x: x[0:4])

In [103]:
years04_05["subprovince code"] = years04_05["subprovince code"].astype("int")

In [104]:
# Now we import the stratum name - code data
strat_name = pd.read_csv(
    "/Users/clarissaache/Documents/IDS 701/stratum0405_v2.csv"
)
strat_name.sample(5)

,province,subprovince code,subprovince
77,Balochistan,4032,Kalat
13,Punjab,1132,Gujrat
71,NWFP,3212,Bannu
42,Sindh,2082,Larkana
44,Sindh,2102,Hyderabad


In [105]:
# MERGE
years04_05_ = pd.merge(years04_05, strat_name, on=["subprovince code"], how='inner', indicator=True)

# Evaluate merge 
years04_05_._merge.value_counts()

both          25457
left_only         0
right_only        0
Name: _merge, dtype: int64

In [106]:
# add year
years04_05_['year']=2004
years04_05_['marital_status']=years04_05_['marital_status'].astype('str')

#fix marital status categories
years04_05_.loc[(years04_05_['marital_status'] == "1:never married"), 'marital_status'] = 1
years04_05_.loc[(years04_05_['marital_status'] == "2:currently married"), 'marital_status'] = 2
years04_05_.loc[(years04_05_['marital_status'] == "5:nikkah soleminised but rukhsati not taken place"), 'marital_status'] = 5
years04_05_.loc[(years04_05_['marital_status'] == "4:widow / widowers"), 'marital_status'] = 3
years04_05_.loc[(years04_05_['marital_status'] == "3:divoced"), 'marital_status'] = 4
years04_05_['marital_status'].value_counts()
years04_05_['marital_status']=years04_05_['marital_status'].astype('int')

# fix column names
years04_05_['idc']=years04_05_['msno']
years04_05_['province']=years04_05_['province_x']

In [107]:
#select relevant columns
years04_05_ = years04_05_[
    [
        "hhcode",
        "idc",
        "age",
        "marital_status",
        "sex",
        "ever_admitted",
        "currently_enrolled",
        "region",
        "province",
        "subprovince code",
        "province",
        "year",
    ]
].copy()

In [108]:
years04_05_.sample(5)

,hhcode,idc,age,marital_status,sex,ever_admitted,currently_enrolled,region,province,subprovince code,province,year
13035,2092002604.0,6,10,1,female,yes,yes,rural,sindh,2092,sindh,2004
20367,3212001608.0,5,7,1,female,no,NaN,rural,nwfp,3212,nwfp,2004
9590,1342002811.0,6,5,1,male,no,NaN,rural,punjab,1342,punjab,2004
25105,4242000212.0,4,10,1,male,no,NaN,rural,balochistan,4242,balochistan,2004
7633,1252001710.0,5,11,1,male,yes,yes,rural,punjab,1252,punjab,2004


In [109]:
years04_05_.to_csv('/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/10_data_cleaning/clean_clarissa/years04_05_.csv', index_label="id")